In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from keras.utils import to_categorical
from keras import models
from keras import layers
from sklearn.feature_extraction.text import CountVectorizer

Using TensorFlow backend.


In [2]:
def read_ontology():
    go = dict()
    obj = None
    with open('go.txt', 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            if line == '[Term]':
                if obj is not None:
                    go[obj['id']] = obj
                obj = dict()
                obj['is_a'] = list() #список прародителй класса
                #obj['part_of'] = list()
                #obj['regulates'] = list()
                continue
            elif line == '[Typedef]':
                obj = None
            else:
                if obj is None:
                    continue
                l = line.split(": ")
                if l[0] == 'id':
                    obj['id'] = l[1]
                elif l[0] == 'is_a':
                    obj['is_a'].append(l[1].split(' ! ')[0])
                elif l[0] == 'name':
                    obj['name'] = l[1]
                elif l[0] == 'namespace':
                    if l[1] == 'biological_process':
                        obj['namespace'] = 'BP'
                    elif l[1] == 'molecular_function':
                        obj['namespace'] = 'MF'
                    elif l[1] == 'cellular_component':
                        obj['namespace'] = 'CC'
                elif l[0] == 'is_obsolete' and l[1] == 'true':
                    obj = None #важно! мы не считываем устаревшие анотации
    if obj is not None:
        go[obj['id']] = obj
    for go_id, val in iter(go.items()): #для каждого класса go создаем поле с его дочерними классами
        if 'children' not in val:
            val['children'] = set()
        for p_id in val['is_a']:
            if p_id in go:
                if 'children' not in go[p_id]: #вдруг родитель класса - та анотация, которую мы не считали по причине того, что она устарела? надо проверить существование p_id
                    go[p_id]['children'] = set()
                go[p_id]['children'].add(go_id)
    return go
    
go = dict()
go = read_ontology()
#model = Sequential()


In [3]:
count = 0
total_len = 0
i = 0
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqIO.FastaIO import SimpleFastaParser
def is_seq_valid(seq):
    if(len(seq)>1002):
        return False
    ignor_symbols = 'BOJUXZ'
    for symbol in seq:
        for ignor_symbol in ignor_symbols:
            if symbol == ignor_symbol:
                return False
    return True

def def_ignore_classes(data):
    all_classes = dict()
    ignor_classes = list()
    for prot in data:
        for prot_class in prot['GO']:
            if prot_class not in all_classes.keys():
                all_classes[prot_class] = 1
            else:
                #print('a')
                curr_value = all_classes[prot_class]
                curr_value+=1
                all_classes[prot_class] = curr_value
    for key, val in iter(all_classes.items()):
        if (val<5):
            ignor_classes.append(key)
    return ignor_classes

def filter_by_class(classes, ignor_classes):
    for prot_class in classes:
        for ignore_prot_class in ignore_classes:
            if(prot_class == ignore_prot_class):
                return False
    return True
    
#from Bio import SeqIO
#record = SeqIO.read("uniprot-reviewed_yes.fasta", "fasta")
#with open("uniprot-reviewed_yes.fasta") as f:
    #for title, seq in SimpleFastaParser(f):
    #for line in f:
        #if(i==40):
           # break
       # i+=1
        #seq = SeqIO(line)
        #print(seq.description)
        #print(seq.id)
        #print(seq.name)
        #line.split("OX=")
        #print(title)

In [4]:
import json

def reduce_data(data, res): #для 74 сек. - 1/20, для 4 минут - 1/5
    size = len(data)
    if(res>=1):
        return False
    print('размер до:\n', size)
    print('остаточная доля белков:\n', res)
    new_data = list()
    new_size = int(res*size)
    for i in range(new_size):
        new_data.append(data[i])
    data.clear()
    return new_data

def reduce_go(go, ignore_classes):
    print('размер GO до: ', len(go))
    all_classes = list()
    keys = go.keys()
    for k in keys:
        all_classes.append(k)
    go_id = 0
    for go_id in all_classes:
        for ig_c in ignore_classes:
            if (go_id == ig_c):
                del go[go_id]
    print('размер GO после: ', len(go))
    return go
                
    

with open('uniprot-go.json', 'r') as f:
    data = json.loads(f.read())
    data = data['annotations']
    data = reduce_data(data, 1/20)
    #prot_amount = len(data)
    print('Размер до обработки: ', len(data))
    ignore_classes = def_ignore_classes(data)
    go = reduce_go(go, ignore_classes)
    invalid_elems = []
    i=0
    for prot in data:
        if (is_seq_valid(prot['sequence']) == False or filter_by_class(prot['GO'], ignore_classes) == False): #or filter_by_class(prot['GO'], ignore_classes) == False
            invalid_elems.append(i)
        i+=1
    print('Количество лишних элементов: ', len(invalid_elems))
    print(invalid_elems[-1])
    j=0
    for i in invalid_elems:
       # if (j>5000):
       #     print(j)
        del data[i-j]
        j+=1
    print('Размер после фильтрации: ', len(data))
#help(data)
#def read_proteins():
    #json.loads('uniprot-go.json')
    
    for i in range(2):
        print(data[i])


размер до:
 294860
остаточная доля белков:
 0.05
Размер до обработки:  14743
размер GO до:  45005
размер GO после:  36651
Количество лишних элементов:  4610
14738
Размер после фильтрации:  10133
{'GO': ['GO:0016020', 'GO:0016020', 'GO:0016021'], 'sequence': 'MLMMVIRIYIHKRLNAKDYNNFYTSIGDFGYTIPGSDAPQYNIESDSEHYLNFGIAHINFNNGTNNQITNPNPEVYTRFYIQMKTPDGQDRIVYEAENANAISFLIRSTYHKPMSGLVGGFRFDPLLGIRVYKDSFVLADELARIQSEFTTVYGNSGINVTENTTNYSVRMYWESAVVVEPGEPSPENPIPWLSNLPNITDGNPENIDGQWGNVTGFNYNPTTKLFNADVIYDGNSYPIRDAVVASNDDSFLTRTEKAYYFTQGGQKFLYLNFSKDIDSFMFKNHYSELKNWVGHALWNLSTGEVNVMEQLKVFQYLKFTEDNELMAYFYIVDTPVDDLISVTSNVAYRYWKPTWFGLGPKEPGPIQNKIITAAQGEFNSIRPTWVKPVYLGSYFAGASGLVIAGSALIKGGSPWFGLGLAGVGILAGGILQYADDNEWLNYDIAQVQKLEPNVNIRSEISEAYFNAYGQMFPSTLGQSLYRISYGQFDQSDLQVISDKSDVITVVFETDGITHTYQKDQIDDTWDGPATEIPLGVDGILPEWALWLITIVIGLFALSNLQKIFDTIKKKPIISIIVVIAILYALTYFNLL', 'id': 'P42536'}
{'id': 'Q5E7H2', 'GO': ['GO:0000166', 'GO:0003746', 'GO:0003746', 'GO:0003746', 'GO:0003924', 'GO:0005525', 'GO:0005525', 'GO:00055

In [ ]:
#all_class = dict()
#print(all_class.keys())
#string = 'hjhhj'
#if string not in all_class.keys():
##    all_class[string] = 1
#print(all_class)
#i = all_class[string]
#i+=1
#all_class[string] = i
#all_class['abc'] = 1
#print(all_class)
#l = list()
#for key, val in iter(all_class.items()):
#    if (val==2):
#        l.append(key)
#print(l)
l = ['01', '02', '03']
for i in l:
    z = int(i)
    print(z)
pos = l.index('02')
l[pos] = '04'
print(l)
import pandas as pd
df = pd.DataFrame({'sequnce': [], 'mf_vector': []})
#df.add({'sequnce1': 1})
df1 = pd.DataFrame({'sequnce': ['iyu'], 'mf_vector': [0]})
df = df.append(df1)
print(df)
p = '01'
l.append(p)
print(l)
print('\n')
i=0
while i<0:
    print('yuyu')
    break
else:
    print('hahaha')

In [5]:
import pandas as pd
def def_mf_branch(go, data):
    mf_branch = list()
    prot_sequences = []
    prot_vectors = []
    #df = pd.dataFrame()
    df = dict.fromkeys(['sequence', 'vector'])
    for go_id in go:
        if (go[go_id]['namespace'] == 'MF'):
            l = go[go_id]['id'].split(':')
            int_id = int(l[1])
            mf_branch.append(int_id)
    print(len(mf_branch))
    branch_size = len(mf_branch)
    for prot in data:
        prot_vect = np.zeros(branch_size, dtype = int)
        pos = None
        for prot_class in prot['GO']:
            l = prot_class.split(':')
            int_id = int(l[1])
            if (mf_branch.count(int_id)==0):
                continue
            pos = mf_branch.index(int_id)
            prot_vect[pos] = 1
        if (pos is not None):
            prot_sequences.append(prot['sequence'])
            prot_vectors.append(prot_vect)
            df['vector'] = prot_vectors
            df['sequence'] = prot_sequences
    return df
    
df = def_mf_branch(go, data)            

9244


In [6]:
model = models.Sequential()
import time

def create_sentence(prot):
    sentence = ''
    for l in prot:
        sentence = sentence+l+' '
    sentence = sentence.rstrip()
    return sentence
    
def text_transform(df):
    j = 0
    for i in df['sequence']:
        df['sequence'][j] = create_sentence(i)
        j+=1

def max_len(proteins):
    max = 1
    for prot in proteins:
        p_size = len(prot)
        if (p_size>max):
            max = p_size
    print('Максимальная длина последовательности: ', max)
    return max

def vectorize_sequences(df):
    three_grams = list()
    sequences = list()
    j=0
    t=0
    z = 0
    max_size = max_len(df['sequence'])
    for prot in df['sequence']:
        l=0
        prot_vect = np.zeros(max_size-2, dtype = int)
        while l<len(prot)-2:
            three_gram = prot[l] + prot[l+1] + prot[l+2]
            if (j<4):
                print('j')
                print(three_gram)
                j+=1
            i=0
            while i < len(three_grams):
                if (three_grams[i] == three_gram):
                    if (t<4):
                        print('t')
                        print(three_grams)
                        print(three_gram)
                        t+=1
                    prot_vect[l] = i+1
                    break
                i+=1
            else:
                if (z<4):
                    print('z')
                    print(three_grams)
                    print(three_gram)
                    z+=1
                three_grams.append(three_gram)
                prot_vect[l] = len(three_grams)
            l+=1
        sequences.append(prot_vect)
    return sequences

start_time = time.time()
df['sequence'] = vectorize_sequences(df)
print("--- %s seconds ---" % (time.time() - start_time))

i=0
while i < 3:
    print(df['sequence'][i])
    i+=1

Максимальная длина последовательности:  1001
j
MTD
z
[]
MTD
j
TDL
z
['MTD']
TDL
j
DLS
z
['MTD', 'TDL']
DLS
j
LSK
z
['MTD', 'TDL', 'DLS']
LSK
t
['MTD', 'TDL', 'DLS', 'LSK', 'SKY', 'KYR', 'YRN', 'RNI', 'NIG', 'IGI', 'GIF', 'IFA', 'FAH', 'AHV', 'HVD', 'VDA', 'DAG', 'AGK', 'GKT', 'KTT', 'TTT']
TTT
t
['MTD', 'TDL', 'DLS', 'LSK', 'SKY', 'KYR', 'YRN', 'RNI', 'NIG', 'IGI', 'GIF', 'IFA', 'FAH', 'AHV', 'HVD', 'VDA', 'DAG', 'AGK', 'GKT', 'KTT', 'TTT', 'TTE', 'TER', 'ERI', 'RIL', 'ILK', 'LKL', 'KLT', 'LTG', 'TGK', 'GKI', 'KIH', 'IHK', 'HKI', 'KIG', 'IGD', 'GDT', 'DTH', 'THD', 'HDG', 'DGS', 'GST', 'STT']
TTT
t
['MTD', 'TDL', 'DLS', 'LSK', 'SKY', 'KYR', 'YRN', 'RNI', 'NIG', 'IGI', 'GIF', 'IFA', 'FAH', 'AHV', 'HVD', 'VDA', 'DAG', 'AGK', 'GKT', 'KTT', 'TTT', 'TTE', 'TER', 'ERI', 'RIL', 'ILK', 'LKL', 'KLT', 'LTG', 'TGK', 'GKI', 'KIH', 'IHK', 'HKI', 'KIG', 'IGD', 'GDT', 'DTH', 'THD', 'HDG', 'DGS', 'GST', 'STT', 'TTD', 'TDF', 'DFM', 'FME', 'MEQ', 'EQE', 'QEA', 'EAE', 'AER', 'ERG', 'RGI', 'GIT', 'ITI', 'T

In [9]:
def save_data(f_seq, f_vect, df): #f_seq - путь к файлу с последовательностями, f_vect - путь к фалу с векторами классов
    with open(f_seq, 'x') as f1:
        for seq in df['sequence']:
            str1 = ''.join(str(e)+',' for e in seq)
            str1 = str1.rstrip(',')
            f1.write(str1 + '\n')
    f1.close()
    with open(f_vect, 'x') as f2:
        for vect in df['vector']:
            str2 = ''.join(str(e)+',' for e in vect)
            str2 = str2.rstrip(',')
            f2.write(str2 + '\n')
    f2.close()
    
def load_data(f_seq, f_vect):
    df = dict.fromkeys(['sequence', 'vector'])
    sequences = list()
    vectors = list()
    with open(f_seq, 'r') as f1:
        #print('enter seq_f')
        #curr_seq = list()
        for line in f1:
            curr_seq = list(map(int, line.split(',')))
            sequences.append(curr_seq)
            df['sequence'] = sequences
    with open(f_vect, 'r') as f2:
        #print('enter vect_f')
        #curr_vect = list()
        for line in f2:
            curr_vect = list(map(int, line.split(',')))
            vectors.append(curr_vect)
            df['vector'] = vectors
    return df

In [10]:
#print("Testing saving and loading data")
#start_time = time.time()
#save_data('proteinsvect.txt', 'vectorsmfbranch.txt', df)
#print("saving data --- %s seconds ---" % (time.time() - start_time))
import time
start_time = time.time()
df1 = load_data('proteinsvect.txt', 'vectorsmfbranch.txt')
print("loading data --- %s seconds ---" % (time.time() - start_time))
#if (df1==df):
#    print('test has been completed')

loading data --- 29.342000246047974 seconds ---


In [29]:
#import libraries
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from keras.utils import to_categorical
from keras import models
from keras import layers as ls
from keras.activations import sigmoid
from sklearn.feature_extraction.text import CountVectorizer
from keras import optimizers as opt
from sklearn.metrics import f1_score
import numpy as np

data_prot = np.asarray(df['sequence'])
max_index = np.amax(data_prot)
print("Max index of a threegram is %i" %max_index)

seq_length = (data_prot.shape)[1] #сейчас это 999. Т.к. колличество триграмм в последовательности длинной 1001 - 999
print('Protein sample have length %i' %seq_length)
vocab_size = max_index+1 #размер словаря это максимально возможный индекс в последовательности + 1

model = models.Sequential()
model.add(ls.Embedding(vocab_size, 128, input_length=seq_length))
model.add(ls.Conv1D(filters = 32, kernel_size = 128, use_bias=True))
model.add(ls.MaxPooling1D(pool_size= 64, stride = 32))
model.add(ls.Dense(32))
model.add(ls.Flatten())
model.add(ls.Dense(9244))
model.add(ls.Activation('sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer=opt.SGD(lr = 0.1, momentum = 0.003),
              metrics=['acc'])
model.summary()

n_samples = len(df['sequence'])
#print(n_samples)
print("We have %i samples" %(len(df['vector'])))


Max index of a threegram is 8000
Protein sample have length 999


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=64, strides=32)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 999, 128)          1024128   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 872, 32)           524320    
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 26, 32)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 26, 32)            1056      
_________________________________________________________________
flatten_3 (Flatten)          (None, 832)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 9244)              7700252   
_________________________________________________________________
activation_3 (Activation)    (None, 9244)              0         
Total para

In [31]:
data_part = (n_samples//10)*8
print("We use %i samples for training" %(data_part))
x_train, y_train, x_test, y_test = df['sequence'][:data_part], df['vector'][:data_part], df['sequence'][data_part:], df['vector'][data_part:]

x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
x_test = np.asarray(x_test)
y_test = np.asarray(y_test)

print("Cheking the shape of target vectors:")
print(x_train.shape)
print(y_train.shape)

#print(x_train[30][0])

model.fit(x_train, y_train)
y_pred = model.predict(x_test)

def approximation(threeshold, y_pred):
    shape = y_pred.shape
    y_flatten = y_pred.flatten()
    size = len(y_flatten)
    i=0
    while (i<size):
        if (y_flatten[i]>=threeshold):
            y_flatten[i] = 1
        else:
            y_flatten[i] = 0
        i+=1
    y_new = y_flatten.reshape(shape)
    y_new = y_new.astype("int32")
    return y_new

y_app = approximation(0.5, y_pred)
test = y_test.flatten()
pred = y_app.flatten()
print("pred")
j=0
while j<9:
    j+=1
    print(pred[j])
print("test")
j=0
while j<9:
    j+=1
    print(test[j])
f1_score(test, pred, average='weighted')

We use 7224 samples for training
Cheking the shape of target vectors:
(7224, 999)
(7224, 9244)
Epoch 1/1
7224/7224 [==============================] - 962s 133ms/step - loss: 0.6871 - acc: 0.5518
pred
0
1
1
1
0
0
0
0
0
test
0
0
0
0
0
0
0
0
0


0.7896750387822402

In [34]:
y_app = approximation(0.6, y_pred)
pred1 = y_app.flatten()
f1_score(test, pred1, average="weighted")

0.9866710023917785

In [35]:
y_app = approximation(0.6, y_pred)
pred1 = y_app.flatten()
f1_score(test, pred1)

0.00023896374076651133